In [ ]:
from google.colab import files

uploaded = files.upload()   # choose D3_WNtrain100k.csv and D2test.csv

Saving D3_WNtrain100k.csv to D3_WNtrain100k.csv
Saving D2test.csv to D2test.csv


In [ ]:
!pip install transformers scikit-learn pandas numpy torch accelerate

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report

import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizerFast, BertForSequenceClassification, get_linear_schedule_with_warmup
from torch.optim import AdamW

In [ ]:
train_df = pd.read_csv("/content/D3_WNtrain100k.csv")
test_df  = pd.read_csv("/content/D2test.csv")

le = LabelEncoder()
y_train = le.fit_transform(train_df['label'].astype(str).values)
y_test  = le.transform(test_df['label'].astype(str).values)
num_labels = len(le.classes_)

tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")
max_len = 40

train_enc = tokenizer(
    train_df['description'].tolist(),
    truncation=True, padding=True, max_length=max_len, return_tensors="pt"
)
test_enc = tokenizer(
    test_df['description'].tolist(),
    truncation=True, padding=True, max_length=max_len, return_tensors="pt"
)

y_train_t = torch.tensor(y_train)
y_test_t  = torch.tensor(y_test)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
class InvoiceDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __len__(self):
        return len(self.labels)
    def __getitem__(self, idx):
        item = {k: v[idx] for k, v in self.encodings.items()}
        item["labels"] = self.labels[idx]
        return item

train_dataset = InvoiceDataset(train_enc, y_train_t)
test_dataset  = InvoiceDataset(test_enc, y_test_t)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader  = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=num_labels
)
model.to(device)

epochs = 3
optimizer = AdamW(model.parameters(), lr=2e-5)

total_steps = len(train_loader) * epochs
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=int(0.1 * total_steps),
    num_training_steps=total_steps
)

Device: cuda


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model.train()
for epoch in range(epochs):
    total_loss = 0
    for batch in train_loader:
        optimizer.zero_grad()
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        scheduler.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}/{epochs} - loss: {total_loss/len(train_loader):.4f}")

Epoch 1/3 - loss: 0.2650
Epoch 2/3 - loss: 0.0019
Epoch 3/3 - loss: 0.0002


In [ ]:
model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for batch in test_loader:
        labels = batch["labels"].to(device)
        inputs = {k: v.to(device) for k, v in batch.items() if k != "labels"}
        outputs = model(**inputs)
        logits = outputs.logits
        preds = torch.argmax(logits, dim=1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

all_preds = np.array(all_preds)
all_labels = np.array(all_labels)

print("Accuracy:", (all_preds == all_labels).mean())
# use only labels present in test set for report
labels_test = np.unique(all_labels)
print(classification_report(
    all_labels,
    all_preds,
    labels=labels_test,
    target_names=le.classes_[labels_test]
))


Accuracy: 1.0
              precision    recall  f1-score   support

  CLASS-1248       1.00      1.00      1.00         1
  CLASS-1249       1.00      1.00      1.00        18
  CLASS-1250       1.00      1.00      1.00        27
  CLASS-1274       1.00      1.00      1.00       142
  CLASS-1294       1.00      1.00      1.00         4
  CLASS-1309       1.00      1.00      1.00         5
  CLASS-1322       1.00      1.00      1.00         4
  CLASS-1376       1.00      1.00      1.00        52
  CLASS-1429       1.00      1.00      1.00         5
  CLASS-1477       1.00      1.00      1.00         7
  CLASS-1522       1.00      1.00      1.00       104
  CLASS-1567       1.00      1.00      1.00        10
  CLASS-1652       1.00      1.00      1.00         2
  CLASS-1688       1.00      1.00      1.00         1
  CLASS-1721       1.00      1.00      1.00        15
  CLASS-1758       1.00      1.00      1.00       144
  CLASS-1770       1.00      1.00      1.00         3
  CLASS-1805 